In [359]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import asyncio

In [20]:
import pandas as pd

In [342]:
driver = webdriver.Chrome()

In [46]:
def iniciarSesion():
    driver.get('https://www.linkedin.com/') # Abrir linkedin
    time.sleep(5)

    username = driver.find_element(By.ID, "session_key")
    username.send_keys("julietaciare77@gmail.com") # Poner correo 

    pword = driver.find_element(By.ID, "session_password")
    pword.send_keys("Many4409")    #Poner tu contraseña
    time.sleep(5)  

    driver.find_element(By.XPATH, "//button[@type='submit']").click() # Tocar el botón de iniciar sesión 

In [343]:
iniciarSesion()

In [24]:
def trabajo(url):
    driver.get(url) ## Abrir la urls de empleos. Por ejemplo: Analista de datos 
    time.sleep(3)
    # Filtrado por fecha
    wait = WebDriverWait(driver, 30)
    button = wait.until(EC.element_to_be_clickable((By.ID, "searchFilter_timePostedRange")))
    button.click()
    
    option = wait.until(EC.presence_of_element_located((By.ID, "timePostedRange-r86400")))
    ## Filtrar por 24 horas
    driver.execute_script("arguments[0].click();", option)

    # Click en mostrar los resultados del filtro
    driver.find_element(By.XPATH, "//button[@type='button']").click()

In [275]:
trabajo(f"https://www.linkedin.com/jobs/search/?currentJobId=3692544132&keywords=data%20science&refresh=true")

In [310]:
linksJobs = []

In [313]:
locationCompany = []

In [314]:
def obtencionLinkJobs(num_pages):
    try: 
        for page in range(2,num_pages+2):
            time.sleep(2)
            jobs = driver.find_element(By.CLASS_NAME ,'jobs-search-results-list')
            jobs_list= jobs.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')
        
            for job in jobs_list:
                all_links = job.find_elements(By.TAG_NAME, "a")
                for a in all_links:
                    if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in linksJobs: 
                        linksJobs.append(a.get_attribute('href')) # agregar los links en la lista
                        locationCompany.append(job.find_element(By.CLASS_NAME,"job-card-container__metadata-item ").text)
                    else:
                        pass
                # scroll 
                driver.execute_script("arguments[0].scrollIntoView();", job)

            
            print(f'Links cargados de la página: {page-1}') # imprimir la página a la cual ha extraídos los links

            driver.find_element(By.XPATH,f"//button[@aria-label='Página {page}']").click() # Pasar a la página siguiente
            time.sleep(3)
    except:
        pass

In [317]:
obtencionLinkJobs(2)

Links cargados de la página: 1
Links cargados de la página: 2


In [318]:
len(linksJobs)

30

In [319]:
len(locationCompany)

30

In [ ]:
pd.DataFrame({'location': locationCompany, 'link': linksJobs})

In [322]:
linksJobs

['https://www.linkedin.com/jobs/view/3719640681/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=iK%2FVw%2Bb3gZe3GJdQeLRspw%3D%3D&trackingId=FaZmJIwJScsfxeYS4RxkKg%3D%3D&trk=flagship3_search_srp_jobs',
 'https://www.linkedin.com/jobs/view/3715813639/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=iK%2FVw%2Bb3gZe3GJdQeLRspw%3D%3D&trackingId=uzDm3z4EsCAkQ9qXfG6p0A%3D%3D&trk=flagship3_search_srp_jobs',
 'https://www.linkedin.com/jobs/view/3719615921/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=iK%2FVw%2Bb3gZe3GJdQeLRspw%3D%3D&trackingId=%2F9UhetGh%2BGneR6sDKpnwPw%3D%3D&trk=flagship3_search_srp_jobs',
 'https://www.linkedin.com/jobs/view/3718996887/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=iK%2FVw%2Bb3gZe3GJdQeLRspw%3D%3D&trackingId=aSitNNXQow3jxhAhCHRWpw%3D%3D&trk=flagship3_search_srp_jobs',
 'https://www.linkedin.com/jobs/view/3719685529/?eBP=CwEAAAGKqhxiqX7zPApPJmGKxXRIyeKxNRc1MrOpwKP6crCyeC2Rp-S0BNVLae658owKimLKZUs42gogEZxRpptsbQkJ2blKlb2Ey4X16aGwS5lpSUt50xju56stJO4c1VwwTnzTMk_Sirz94SFUDzAuP_FDtt3hWpvT8QHw277f28bXHSmsMpB

In [324]:
titleJobs = []
companyName = []
employeeRank = []
dateJobs = []
workType = []

In [347]:
skill = [] 

In [325]:
def getDate(time):
    fecha_hora_actual = datetime.now()
    if time.split()[1] == 'horas' or time.split()[1] == 'hora':
        hora = time.split()[0]
        nueva_fecha = fecha_hora_actual - timedelta(hours=int(hora))
        fecha_formateada = nueva_fecha.strftime("%d/%m/%Y")
        return fecha_formateada
    if time.split()[1] == 'minutos' or time.split()[1] == 'minuto':
        minutos = time.split()[0]
        nueva_fecha = fecha_hora_actual - timedelta(minutes=int(minutos))
        fecha_formateada = nueva_fecha.strftime("%d/%m/%Y")
        return fecha_formateada
    if time.split()[1] == 'segundos' or time.split()[1] == 'segundo':
        segundos = time.split()[0]
        nueva_fecha = fecha_hora_actual - timedelta(seconds=int(segundos))
        fecha_formateada = nueva_fecha.strftime("%d/%m/%Y")
        return fecha_formateada

In [330]:
async def Extractdata():
    i = 0
    j = 1
    for i in range(len(linksJobs)):
        try:
            driver.get(linksJobs[i])
            i = i+1
            time.sleep(2)
        except:
            pass
        
        # Find the general information of the job offers
        
        contents = driver.find_elements(By.CLASS_NAME,'p5')
        for content in contents:
      
            prue = content.find_element(By.CLASS_NAME,'jobs-unified-top-card__primary-description')
            funcioTi = lambda:content.find_element(By.TAG_NAME,"h1").text if content.find_element(By.TAG_NAME,"h1").text else None
            titleJobs.append(funcioTi())
            funcionCom = lambda: prue.find_element(By.CLASS_NAME,'app-aware-link').text if prue.find_element(By.CLASS_NAME,'app-aware-link').text else None
            companyName.append(funcionCom())
            funcionWork = lambda: driver.find_elements(By.CLASS_NAME,"jobs-unified-top-card__job-insight")[0].text if driver.find_elements(By.CLASS_NAME,"jobs-unified-top-card__job-insight")[0].text else None
            workType.append(funcionWork())
            funcionemplo = lambda:driver.find_elements(By.CLASS_NAME,"jobs-unified-top-card__job-insight")[1].text if driver.find_elements(By.CLASS_NAME,"jobs-unified-top-card__job-insight")[1].text else None
            employeeRank.append(funcionemplo())
            funci =  lambda: getDate(content.find_element(By.TAG_NAME,'strong').text.split('hace')[1].strip()) if getDate(content.find_element(By.TAG_NAME,'strong').text.split('hace')[1].strip()) else None
            dateJobs.append(funci())
            j+=1
        

                

In [350]:
async def ExtractSkill():
    for i in range(len(linksJobs)):
        driver.get(linksJobs[i])
        i = i+1
        time.sleep(2)
        try:
            fil = driver.find_element(By.CLASS_NAME,f"job-details-how-you-match-card__container")
            fil.find_element(By.TAG_NAME,'button').click()
            time.sleep(5)
            filtro = driver.find_elements(By.CLASS_NAME,"job-details-skill-match-status-list")[0].text.replace("\n", "/").split('/')
            lista1 = [filtro for filtro in filtro if filtro != "Añadir"]
            skill.append(lista1)
        except NoSuchElementException:
            skill.append('Dato no encontrado')

In [351]:
await ExtractSkill()

In [352]:
len(skill)

30

In [331]:
await Extractdata()

In [353]:
df = pd.DataFrame({'titleJobs': titleJobs,'comanyName': companyName,'companyLocation': locationCompany,'employeeRank': employeeRank,
'workType': workType, 'dateJobs': dateJobs,'skill': skill, 'link': linksJobs})

In [354]:
df

,titleJobs,comanyName,companyLocation,employeeRank,workType,dateJobs,skill,link
0,Data Scientist,TinyBytes,Argentina (En remoto),De 11 a 50 empleados,Jornada completa,18/09/2023,"[Aprendizaje automático, Ciencia de datos, Cie...",https://www.linkedin.com/jobs/view/3719640681/...
1,Gerente de operaciones,La Fonte D'Oro,Buenos Aires y alrededores (Presencial),De 201 a 500 empleados,Jornada completa,18/09/2023,"[Analítica de datos, Aptitudes para la supervi...",https://www.linkedin.com/jobs/view/3715813639/...
2,Research Analyst,TheMindCo,Buenos Aires y alrededores (Híbrido),De 11 a 50 empleados,Jornada completa,18/09/2023,"[Análisis de datos, Microsoft Excel, Microsoft...",https://www.linkedin.com/jobs/view/3719615921/...
3,Research Analyst Sr,Happy Together,Buenos Aires y alrededores (Híbrido),De 51 a 200 empleados,Jornada completa,18/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3718996887/...
4,ANALISTA PLANIFICACION ESTRATEGICA DE PRECIOS,YPF SA,Buenos Aires y alrededores (Híbrido),Más de 10.001 empleados · Petróleo y gas,Jornada completa · Intermedio,18/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3719685529/...
5,Android + iOS Developer,Clevertech,"Córdoba y alrededores, Argentina (En remoto)",De 501 a 1.000 empleados · Servicios de TI y c...,Jornada completa · Intermedio,18/09/2023,"[Metodologías ágiles, Desarrollo de Android, D...",https://www.linkedin.com/jobs/view/3723699056/...
6,Senior Python Engineer,Clevertech,"Córdoba y alrededores, Argentina (En remoto)",De 501 a 1.000 empleados · Servicios de TI y c...,Jornada completa · Intermedio,18/09/2023,"[Ingeniería, Microsoft Office, Python, Resoluc...",https://www.linkedin.com/jobs/view/3723721025/...
7,Senior Python Engineer,Clevertech,"Provincia de Buenos Aires, Argentina (En remoto)",De 501 a 1.000 empleados · Servicios de TI y c...,Jornada completa · Intermedio,18/09/2023,"[Ingeniería, Microsoft Office, Python, Resoluc...",https://www.linkedin.com/jobs/view/3723717665/...
8,iOS Developer,Clevertech,Buenos Aires y alrededores (En remoto),De 501 a 1.000 empleados · Servicios de TI y c...,Jornada completa · Intermedio,18/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3723692767/...
9,Peformance Engineer,Consensys,Argentina (En remoto),De 501 a 1.000 empleados · Desarrollo de software,Jornada completa · Sin experiencia,18/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3719662565/...


Transform